In [1]:
import cv2
import mediapipe as mp
import time

# MediaPipe Pose 모델 로드
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# 카운터 초기화
squat_count = 0
is_squatting = False  # 스쿼트 중인지 여부
squat_start_time = 0  # 스쿼트 시작 시간
cooldown_duration = 1.5  # 스쿼트 후 일정 시간 동안 다음 동작 무시
reset_duration = 2.0  # 스쿼트 관절이 3초 동안 감지되지 않으면 초기화
display_duration = 2000  # 2초 동안 메시지 표시

# is_squatting_pose 함수 업데이트
def is_squatting_pose(keypoints):
    # 어깨, 엉덩이, 무릎, 발목의 y-좌표를 확인하여 스쿼트 동작 판정
    left_shoulder_y = keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
    right_shoulder_y = keypoints[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
    left_hip_y = keypoints[mp_pose.PoseLandmark.LEFT_HIP.value].y
    right_hip_y = keypoints[mp_pose.PoseLandmark.RIGHT_HIP.value].y
    left_knee_y = keypoints[mp_pose.PoseLandmark.LEFT_KNEE.value].y
    right_knee_y = keypoints[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
    
    # 스쿼트 동작 판정 조건
    is_preparing_squat = (
        left_knee_y < left_hip_y and
        right_knee_y < right_hip_y and
        left_shoulder_y < left_hip_y and
        right_shoulder_y < right_hip_y
    )

    return is_preparing_squat

# 동영상 파일 경로
video_path = 'C://Users//dokgo//Desktop//final//1s.mp4'

# 웹캠으로부터 비디오 스트림 읽기
cap = cv2.VideoCapture(video_path)

# 비디오 프레임 크기 조정
target_width, target_height = 1024, 768

while cap.isOpened():
    # 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (target_width, target_height))

    # Mediapipe Pose 적용
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    # 감지된 포즈 점 찾기
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # 랜드마크 좌표 시각화
        for lm in landmarks:
            h, w, _ = frame.shape
            cx, cy = int(lm.x * w), int(lm.y * h)
            cv2.circle(frame, (cx, cy), 5, (255, 0, 0), cv2.FILLED)

        # 랜드마크를 선으로 연결하여 시각화
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 0)))



        # 여기에서 is_squatting_pose 함수를 사용하여 스쿼트 동작인지 확인
        if is_squatting_pose(landmarks):
            # 스쿼트 동작이 감지되면 is_squatting을 True로 설정하고 시작 시간 기록
            if not is_squatting:
                is_squatting = True
                squat_start_time = time.time()
                print("스쿼트 동작 시작")
                squat_count += 1

                # 스쿼트 중인 동안 표시할 내용 추가 가능
                cv2.putText(frame, "Squatting", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # 스쿼트 동작이 감지되지 않으면 is_squatting을 False로 설정하고 일정 시간 동안 동작 무시
        elif is_squatting and time.time() - squat_start_time > cooldown_duration:
            is_squatting = False
            print("스쿼트 동작 끝. 현재 스쿼트 카운트:", squat_count)


            # 특정 메시지를 2초 동안 표시
            cv2.putText(frame, "Good", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, "{} Kcal".format(round(squat_count * 0.7)), (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.imshow("LWBM", frame)
            cv2.waitKey(display_duration)
            
       
            
        # 스쿼트 중이 아닐 때 표시할 내용 추가 가능
        else:
            cv2.putText(frame, "Preparation Pose", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 카운트 표시
    cv2.putText(frame, f"Squat Count: {squat_count}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # 화면에 출력
    cv2.imshow("LWBM", frame)

    # 종료 조건
    if cv2.waitKey(1) & 0xFF == 27:  # ESC 키를 누르면 종료
        break

# 종료
cap.release()
cv2.destroyAllWindows()


스쿼트 동작 시작
스쿼트 동작 끝. 현재 스쿼트 카운트: 1
스쿼트 동작 시작
스쿼트 동작 끝. 현재 스쿼트 카운트: 2
스쿼트 동작 시작
스쿼트 동작 끝. 현재 스쿼트 카운트: 3
스쿼트 동작 시작
스쿼트 동작 끝. 현재 스쿼트 카운트: 4
스쿼트 동작 시작
스쿼트 동작 끝. 현재 스쿼트 카운트: 5
